In [13]:
%pip install tensorflow-macos
%pip install tensorflow-metal


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. Primero leemos el archivo csv entregado por nuestros compañeros del TEC 

In [14]:
import pandas as pd

df = pd.read_csv("/Users/S340/Documents/Octavo/Analítica de datos /airbnb-proyecto2-20252/airbnb_limpio.csv")

print("Dimensiones del dataset:", df.shape)
df.head()




Dimensiones del dataset: (5562, 80)


,id,source,name,host_id,host_name,host_since,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,...,has_free_street_parking,has_private_entrance,has_essentials,has_heating,has_wifi,has_pets_allowed,has_hot_water,has_self_check_in,has_freezer,has_exercise_equipment
0,18744501,city scrape,"""Artist´s Creative Residence"" 100m² im Zentrum",129635321,Sylvia,2017-05-09,f,1.0,1.0,"['email', 'phone']",...,1,1,1,1,1,0,1,0,0,0
1,23356842,city scrape,"""Bohemian Residency"" (Central & Quiet) * * * * *",150173398,Vincent,2017-09-11,t,2.0,3.0,"['email', 'phone']",...,1,1,1,1,1,0,1,1,0,0
2,819658084391291386,city scrape,"""Feel at Home"" Flat at the Lerchenauer See",29225873,Skandar,2015-03-12,f,1.0,1.0,"['email', 'phone']",...,0,0,1,1,1,1,1,0,0,1
3,34677963,city scrape,"""Little Star"" Schlafoase im Zentrum",28482431,Adriana,2015-02-28,f,5.0,5.0,"['email', 'phone']",...,0,1,1,1,1,0,1,1,0,0
4,34431776,city scrape,"""Moonlight"" Schlafoase mitten im Szenenviertel",28482431,Adriana,2015-02-28,f,5.0,5.0,"['email', 'phone']",...,0,1,1,1,1,0,1,1,0,0


 ## 2. Selección de variables numéricas y definición de X e y

Para entrenar la red neuronal únicamente utilizamos variables numéricas, por lo cual filtramos el dataset.  
Luego definimos:

- **X** : Variables predictoras  
- **y** : Variable objetivo (`recommended`)



In [15]:
df["recommended"] = df["host_is_superhost"].map({"t": 1, "f": 0})

df["recommended"].value_counts()

numeric_df = df.select_dtypes(include=["int64", "float64"])

# Incluir también la variable objetivo
numeric_df["recommended"] = df["recommended"]

numeric_df.head()

,id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,...,has_private_entrance,has_essentials,has_heating,has_wifi,has_pets_allowed,has_hot_water,has_self_check_in,has_freezer,has_exercise_equipment,recommended
0,18744501,129635321,1.0,1.0,48.159077,11.569463,5,1.0,2.0,3.0,...,1,1,1,1,0,1,0,0,0,0
1,23356842,150173398,2.0,3.0,48.146576,11.603159,8,2.0,3.0,6.0,...,1,1,1,1,0,1,1,0,0,1
2,819658084391291386,29225873,1.0,1.0,48.196742,11.533000,3,1.0,1.0,1.0,...,0,1,1,1,1,1,0,0,1,0
3,34677963,28482431,5.0,5.0,48.130140,11.578380,2,1.0,1.0,1.0,...,1,1,1,1,0,1,1,0,0,0
4,34431776,28482431,5.0,5.0,48.130810,11.578320,2,1.0,1.0,1.0,...,1,1,1,1,0,1,1,0,0,0


In [16]:
X = numeric_df.drop(columns=["recommended"])
y = numeric_df["recommended"]

## 3. Luego dividimos los datos en entrenamiento y prueba usando train_test_split.
Al aplicar estratificación, nos aseguramos de que la proporción entre superhosts y no-superhosts se mantenga igual en ambos subconjuntos.


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape

((4449, 63), (1113, 63))

## 4. Escalamiento de las variables numéricas

Los modelos basados en redes neuronales requieren que los datos estén en la misma escala.  
Por lo tanto, aplicamos **StandardScaler** para normalizar las variables numéricas.

Usamos:
- `fit_transform`: sobre el conjunto de entrenamiento  
- `transform` : sobre el conjunto de prueba  

Esto evita fugas de información (*data leakage*).

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 5.Configuración de MLflow

Se importan las librerías requeridas y se registra el experimento “clasificacion_superhost_nn” en MLflow, donde se guardarán los parámetros y métricas de todas las ejecuciones del modelo.

In [ ]:

import numpy as np

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report,
)

from tensorflow import keras
from tensorflow.keras import layers

import mlflow
import mlflow.keras

# crear el experimento en MLflow
mlflow.set_experiment("clasificacion_superhost_nn")

<Experiment: artifact_location='file:///Users/S340/Documents/Octavo/Anali%CC%81tica%20de%20datos%20/airbnb-proyecto2-20252/Etapa4_Modelos_Clasificacion/mlruns/589417676544699594', creation_time=1764601242511, experiment_id='589417676544699594', last_update_time=1764601242511, lifecycle_stage='active', name='clasificacion_superhost_nn', tags={}>

## 6. Construcción paramétrica del modelo

Se define la función build_nn_model, que crea un modelo de red neuronal configurable mediante hiperparámetros (capas, neuronas, dropout y learning rate). Esto permite generar y comparar múltiples arquitecturas bajo el mismo conjunto de datos.

In [20]:
def build_nn_model(
    input_dim,
    n_neuronas_capa1=64,
    n_neuronas_capa2=32,
    learning_rate=1e-3,
    usar_dropout=False,
    dropout_rate=0.3,
):
    """
    Construye y compila una red neuronal densa para clasificación binaria.
    Los hiperparámetros clave (neuronas, learning_rate, dropout) se pasan como argumentos.
    """
    model = keras.Sequential(name="nn_clf_superhost")

    # Capa de entrada + primera capa oculta
    model.add(
        layers.Dense(
            n_neuronas_capa1,
            activation="relu",
            input_shape=(input_dim,),
            name="dense_1",
        )
    )

    # Dropout opcional
    if usar_dropout:
        model.add(layers.Dropout(dropout_rate, name="dropout_1"))

    # Segunda capa oculta
    model.add(
        layers.Dense(
            n_neuronas_capa2,
            activation="relu",
            name="dense_2",
        )
    )

    if usar_dropout:
        model.add(layers.Dropout(dropout_rate, name="dropout_2"))

    # Capa de salida (clasificación binaria)
    model.add(
        layers.Dense(
            1,
            activation="sigmoid",
            name="output",
        )
    )

    # Optimizador
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

## 7. Función para ejecutar un experimento con MLflow

Definimos la función run_experimento_nn, que permite entrenar un modelo de red neuronal usando distintos hiperparámetros y registrar todo el proceso en MLflow.
Dentro de cada experimento se:
	- Guardan los hiperparámetros utilizados.
	- Construye el modelo con esos parámetros.
	- Entrena y evalúa la red neuronal.
	- Registra las métricas obtenidas para poder comparar resultados entre configuraciones.



In [ ]:
def run_experimento_nn(nombre_run, params):
    """
    Entrena un modelo de red neuronal con los hiperparámetros indicados en `params`,
    evalúa sobre el set de prueba y registra todo en MLflow.
    

    """
    input_dim = X_train_scaled.shape[1]

    with mlflow.start_run(run_name=nombre_run):
        # 1. Log de hiperparámetros
        mlflow.log_params(params)

        # 2. Construir modelo
        model = build_nn_model(
            input_dim=input_dim,
            n_neuronas_capa1=params["n_neuronas_capa1"],
            n_neuronas_capa2=params["n_neuronas_capa2"],
            learning_rate=params["learning_rate"],
            usar_dropout=params["usar_dropout"],
            dropout_rate=params["dropout_rate"],
        )

        # 3. Entrenamiento
        history = model.fit(
            X_train_scaled,
            y_train,
            validation_split=0.2,
            epochs=params["epochs"],
            batch_size=params["batch_size"],
            verbose=0,  
        )

        # 4. Predicciones en test
        y_proba = model.predict(X_test_scaled).ravel()
        y_pred = (y_proba > 0.5).astype("int32")

        # 5. Métricas de evaluación
        acc = accuracy_score(y_test, y_pred)
        prec, rec, f1, _ = precision_recall_fscore_support(
            y_test, y_pred, average="binary"
        )

        prec_macro, rec_macro, f1_macro, _ = precision_recall_fscore_support(
            y_test, y_pred, average="macro"
        )

        # 6. Log de métricas en MLflow
        mlflow.log_metric("accuracy_test", acc)
        mlflow.log_metric("precision_binaria_test", prec)
        mlflow.log_metric("recall_binaria_test", rec)
        mlflow.log_metric("f1_binaria_test", f1)
        mlflow.log_metric("precision_macro_test", prec_macro)
        mlflow.log_metric("recall_macro_test", rec_macro)
        mlflow.log_metric("f1_macro_test", f1_macro)

        mlflow.log_metric("loss_final_train", history.history["loss"][-1])
        mlflow.log_metric(
            "loss_final_val", history.history["val_loss"][-1]
        )

        # 7. Guardar el modelo como artefacto
        mlflow.keras.log_model(model, artifact_path="modelo_nn")

        
        print(f"=== Run: {nombre_run} ===")
        print(f"Accuracy test:  {acc:.3f}")
        print(f"Precision (1):  {prec:.3f}")
        print(f"Recall (1):     {rec:.3f}")
        print(f"F1-score (1):   {f1:.3f}")
        print("-------------------------------")

        # Devolver métricas 
        return {
            "run_name": nombre_run,
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1": f1,
        }

## 8. Definición de los experimentos

En esta sección definimos una lista de experimentos, donde cada una modifica ciertos hiperparámetros de la red neuronal, como el número de neuronas, la tasa de aprendizaje, la cantidad de épocas o el uso de dropout.

In [22]:
experimentos = [
    {
        "nombre": "Modelo Base",
        "n_neuronas_capa1": 64,
        "n_neuronas_capa2": 32,
        "learning_rate": 1e-3,
        "epochs": 50,
        "batch_size": 64,
        "usar_dropout": False,
        "dropout_rate": 0.0,
    },
    {
        "nombre": "Modelo con mas neuronas",
        "n_neuronas_capa1": 128,
        "n_neuronas_capa2": 64,
        "learning_rate": 1e-3,
        "epochs": 50,
        "batch_size": 64,
        "usar_dropout": True,
        "dropout_rate": 0.3,
    },
    {
        "nombre": "Modelo con mas epochs y menor lr",
        "n_neuronas_capa1": 64,
        "n_neuronas_capa2": 32,
        "learning_rate": 5e-4,
        "epochs": 100,
        "batch_size": 64,
        "usar_dropout": True,
        "dropout_rate": 0.3,
    },
    {
        "nombre": "Modelo con batch pequeño",
        "n_neuronas_capa1": 64,
        "n_neuronas_capa2": 32,
        "learning_rate": 1e-3,
        "epochs": 50,
        "batch_size": 32,
        "usar_dropout": False,
        "dropout_rate": 0.0,
    },
]

## 9.  Ejecución de los experimentos

Aquí corremos cada configuración de hiperparámetros definida antes.
Por cada experimento entrenamos un modelo,registramos métricas y parámetros en MLflow y guardamos los resultados para compararlos al final.

In [ ]:
resultados = []

for cfg in experimentos:
    nombre_run = cfg["nombre"]
    params = {k: v for k, v in cfg.items() if k != "nombre"}

    res = run_experimento_nn(nombre_run, params)
    resultados.append(res)

resultados

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


2025/12/01 10:10:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/01 10:10:56 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/12/01 10:11:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


=== Run: Modelo Base ===
Accuracy test:  0.806
Precision (1):  0.568
Recall (1):     0.526
F1-score (1):   0.546
-------------------------------
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


2025/12/01 10:11:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/01 10:11:30 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/12/01 10:11:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

2025/12/01 10:12:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/01 10:12:32 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/12/01 10:12:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


=== Run: Modelo con mas epochs y menor lr ===
Accuracy test:  0.814
Precision (1):  0.667
Recall (1):     0.324
F1-score (1):   0.436
-------------------------------
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


2025/12/01 10:13:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/01 10:13:23 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/12/01 10:13:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


=== Run: Modelo con batch pequeño ===
Accuracy test:  0.824
Precision (1):  0.673
Recall (1):     0.401
F1-score (1):   0.503
-------------------------------


[{'run_name': 'Modelo Base',
  'accuracy': 0.8059299191374663,
  'precision': 0.5676855895196506,
  'recall': 0.5263157894736842,
  'f1': 0.5462184873949579},
 {'run_name': 'Modelo con mas neuronas',
  'accuracy': 0.8122192273135669,
  'precision': 0.6826923076923077,
  'recall': 0.2874493927125506,
  'f1': 0.4045584045584046},
 {'run_name': 'Modelo con mas epochs y menor lr',
  'accuracy': 0.8140161725067385,
  'precision': 0.6666666666666666,
  'recall': 0.32388663967611336,
  'f1': 0.4359673024523161},
 {'run_name': 'Modelo con batch pequeño',
  'accuracy': 0.8238993710691824,
  'precision': 0.673469387755102,
  'recall': 0.4008097165991903,
  'f1': 0.5025380710659898}]

## 10. Imprimimos los resultados en una tabla, mas organizada.


In [24]:
import pandas as pd

df_resultados = pd.DataFrame(resultados)
df_resultados

,run_name,accuracy,precision,recall,f1
0,Modelo Base,0.805930,0.567686,0.526316,0.546218
1,Modelo con mas neuronas,0.812219,0.682692,0.287449,0.404558
2,Modelo con mas epochs y menor lr,0.814016,0.666667,0.323887,0.435967
3,Modelo con batch pequeño,0.823899,0.673469,0.400810,0.502538
